In [27]:
# 12/02/2024
# CSC354 – Assignment1 – ML – Concept Learning
# YAHYA  IRFAN MUFTI
# FA21-BSE-044
'''Write a python program (preferably an ipython notebook) to implement Candidate-Elimination algorithm. 
Give the following training examples as input to the program. The program must output the ordered list of all 
hypotheses consistent with all the training examples'''


def get_domains(examples):
    d = [set() for i in examples[0]] 
    for x in examples:
        for i, xi in enumerate(x):
            d[i].add(xi)
    return [list(sorted(x)) for x in d]

def g_0(n):
    return ('?',)*n

def s_0(n):
    return ('null',)*n

def more_general(h1, h2):
    more_general_parts = []
    for x, y in zip(h1, h2):
        mg = x == '?' or (x != 'null' and (x == y or y == 'null'))
        more_general_parts.append(mg)
    return all(more_general_parts) 

def consistent(hypothesis, example):
    return more_general(hypothesis, example)

def min_generalizations(h, x):
    h_new = list(h)
    for i in range(len(h)):
        if not consistent(h[i:i+1], x[i:i+1]):
            if h[i] != 'null':
                h_new[i] = '?'
            else:
                h_new[i] = x[i]
    return [tuple(h_new)]

def generalize_S(x, G, S):
    S_prev = list(S)
    for s in S_prev:
        if s not in S:
            continue
        if not consistent(s, x):
            S.remove(s)
            Splus = min_generalizations(s, x)
            S.update([h for h in Splus if any([more_general(g, h) for g in G])])
            S.difference_update([h for h in S if any([more_general(h, h1) for h1 in S if h != h1])])
    return S

def min_specializations(h, domains, x):
    results = []
    for i in range(len(h)):
        if h[i] == '?':
            for val in domains[i]:
                if x[i] != val:
                    h_new = h[:i] + (val,) + h[i+1:]
                    results.append(h_new)
        elif h[i] != 'null':
            h_new = h[:i] + ('null',) + h[i+1:]
            results.append(h_new)
    return results

def specialize_G(x, domains, G, S):
    G_prev = list(G)
    for g in G_prev:
        if g not in G:
            continue
        if consistent(g, x):
            G.remove(g)
            Gminus = min_specializations(g, domains, x)
            G.update([h for h in Gminus if any([more_general(h, s) for s in S])])
            G.difference_update([h for h in G if any([more_general(g1, h) for g1 in G if h != g1])])
    return G

def candidate_elimination(examples):
    domains = get_domains(examples)[:-1] 
    G = set([g_0(len(domains))])
    S = set([s_0(len(domains))])
    i = 0
    print('All the hypotheses in General and Specific boundary are:\n')
    print('\n G[{0}]:'.format(i), G)
    print('\n S[{0}]:'.format(i), S)
    for xcx in examples:
        i += 1
        x, cx = xcx[:-1], xcx[-1]  
        if cx == 'yes': 
            G = {g for g in G if consistent(g, x)}
            S = generalize_S(x, G, S)
        else: 
            S = {s for s in S if not consistent(s, x)}
            G = specialize_G(x, domains, G, S)
        print('\n S[{0}]:'.format(i), S)
        print('\n G[{0}]:'.format(i), G)
    return

# Define training examples
examples = [
    ("big", "red", "circle", "no"),
    ("small", "red", "triangle", "no"),
    ("small", "red", "circle", "yes"),
    ("big", "blue", "circle", "no"),
    ("small", "blue", "circle", "yes")
]

candidate_elimination(examples)


All the hypotheses in General and Specific boundary are:


 G[0]: {('?', '?', '?')}

 S[0]: {('null', 'null', 'null')}

 S[1]: {('null', 'null', 'null')}

 G[1]: {('?', '?', 'triangle'), ('small', '?', '?'), ('?', 'blue', '?')}

 S[2]: {('null', 'null', 'null')}

 G[2]: {('small', '?', 'circle'), ('?', 'blue', '?'), ('big', '?', 'triangle')}

 S[3]: {('small', 'red', 'circle')}

 G[3]: {('small', '?', 'circle')}

 S[4]: {('small', 'red', 'circle')}

 G[4]: {('small', '?', 'circle')}

 S[5]: {('small', '?', 'circle')}

 G[5]: {('small', '?', 'circle')}
